## 피마 인디언 당뇨병 예측

In [15]:
import numpy as np
import pandas as pd

- 데이터 전처리 및 탐색

In [16]:
df = pd.read_csv('pima-indians-diabetes.csv', skiprows=9, header=None)
df.head()

# skiprows=9 : 위에서부터 몇줄 생략
# header=None : 데이터에 헤드정보가 없는 경우


,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [17]:
# 1. Number of times pregnant
# 2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
# 3. Diastolic blood pressure (mm Hg)
# 4. Triceps skin fold thickness (mm)
# 5. 2-Hour serum insulin (mu U/ml)
# 6. Body mass index (weight in kg/(height in m)^2)
# 7. Diabetes pedigree function
# 8. Age (years)
# 9. Class variable (0 or 1)

df.columns = ['P', 'G', 'BP', 'S', 'I', 'BMI', 'D', 'Age', 'Target']
df.head()

,P,G,BP,S,I,BMI,D,Age,Target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [18]:
# 결측치 확인 
df.isna().sum().sum()


0

In [19]:
# X, y 데이터를 ndarray 로 추출
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X.shape, y.shape                                 

((768, 8), (768,))

In [20]:
# y 값의 분포
np.unique(y, return_counts=True)

(array([0, 1], dtype=int64), array([500, 268], dtype=int64))

- Train/Test 데이터셋 분리

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,y, stratify=y, test_size=0.2, random_state=2023
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

- GridSearchCV - Decision Tree

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dtc = DecisionTreeClassifier(random_state=2023)
params = {
    'max_depth': [2, 5, 8],
    'min_samples_split': [2, 3, 4]
}
grid_dt = GridSearchCV(dtc,params, scoring='accuracy', cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2023),
             param_grid={'max_depth': [2, 5, 8],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [25]:
grid_dt.best_params_

{'max_depth': 5, 'min_samples_split': 2}

In [26]:
dtc = DecisionTreeClassifier(random_state=2023)
params = {
    'max_depth': [4, 5, 6],
    'min_samples_split': [2, 3, 4]
}
grid_dt = GridSearchCV(dtc,params, scoring='accuracy', cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2023),
             param_grid={'max_depth': [4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [27]:
grid_dt.best_params_

{'max_depth': 6, 'min_samples_split': 3}

- best estimator로 예측 평가

In [28]:
best_dt = grid_dt.best_estimator_
pred = best_dt.predict(X_test)
rf = pd.DataFrame({'y 실제값': y_test, 'y 예측값': pred})
rf.head()

,y 실제값,y 예측값
0,0,0
1,1,1
2,0,0
3,1,1
4,1,1


In [29]:
best_dt.score(X_test, y_test)

0.7207792207792207

- 특정 데이터에 대해 적용

In [30]:
X_test[10], y_test[10]

(array([  0.   , 101.   ,  62.   ,   0.   ,   0.   ,  21.9  ,   0.336,
         25.   ]),
 0)

In [31]:
test_data, test_target = X_test[10], y_test[10]

In [32]:
#result = best_dt.predict(test_data)

# 에러 => Expected 2D array, got 1D array instead: 2차원 배열이 들어올 줄 알았는데 1차원이 들어와서 에러

ValueError: Expected 2D array, got 1D array instead:
array=[  0.    101.     62.      0.      0.     21.9     0.336  25.   ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [36]:
# 정상코드
result = best_dt.predict(test_data.reshape(1, -1))[0]
result

0